In [12]:
#convert onnx to pytorch
import onnx
from onnx2pytorch import ConvertModel

onnx_model = onnx.load('/home/aya/Desktop/Kitti-ObjectDetection/best.onnx')
pytorch_model = ConvertModel(onnx_model)
#put the model on evaluation mode
model=pytorch_model
model.eval()

/home/aya/.local/lib/python3.10/site-packages/onnx2pytorch/convert/layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))
/home/aya/.local/lib/python3.10/site-packages/onnx2pytorch/convert/attribute.py:101: UserWarning: Pytorch's interpolate uses no coordinate_transformation_mode=asymmetric. Result might differ.
  warnings.warn(
/home/aya/.local/lib/python3.10/site-packages/onnx2pytorch/operations/resize.py:16: UserWarning: Pytorch's interpolate uses no cubic_coeff_a. Result might differ.
  warnings.warn(
/home/aya/.local/lib/python3.10/site-packages/

ConvertModel(
  (Conv_/model.0/conv/Conv_output_0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Sigmoid_/model.0/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.0/act/Mul_output_0): mul()
  (Conv_/model.1/conv/Conv_output_0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Sigmoid_/model.1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.1/act/Mul_output_0): mul()
  (Conv_/model.2/cv1/conv/Conv_output_0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (Sigmoid_/model.2/cv1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.2/cv1/act/Mul_output_0): mul()
  (Constant_onnx::Split_140): Constant(constant=tensor([16, 16]))
  (Split_/model.2/Split_output_0): Split()
  (Conv_/model.2/m.0/cv1/conv/Conv_output_0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Sigmoid_/model.2/m.0/cv1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.2/m.0/cv1/act/Mul_output_0): mul()
  (Conv_/model.2/m.0/cv2/conv/Conv_output_0): Conv2d(16, 16, kerne

In [13]:
import numpy as np
from PIL import Image
from io import BytesIO
import torch

from art.estimators.object_detection.pytorch_yolo import PyTorchYolo
from art.attacks.evasion import ProjectedGradientDescent

import cv2
import matplotlib
import matplotlib.pyplot as plt

from ultralytics import YOLO

from torchvision import datasets, transforms

In [14]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
pretrained_model = "data/lenet_mnist_model.pth"
use_cuda=True

In [15]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# Initialize the network

CUDA Available:  True


In [ ]:
import os
import cv2
import numpy as np
x_train=[]
path="/home/aya/Desktop/Kitti-ObjectDetection/kitti-YOLOFormat/train/images/"
for j in os.listdir(path):
    img=cv2.imread(path+j)
    img=cv2.resize(img,dsize=(375, 1244))
    x=img = np.asarray(img)
    x_train.append(x)

y_train=[]
path="/home/aya/Desktop/Kitti-ObjectDetection/kitti-YOLOFormat/train/labels/"
for j in os.listdir(path):
    y=np.loadtxt(path+j)
    y_train.append(y)

In [16]:
!pip install torch-kitti

Defaulting to user installation because normal site-packages is not writeable


In [16]:
#Load the dataset
from torchvision.transforms import Compose, RandomCrop, ToTensor

from torch_kitti.depth_completion import KittiDepthCompletionDataset
from torch_kitti.transforms import ApplyToFeatures

class SeqKittiDepthCompletionDataset(KittiDepthCompletionDataset):
    def __init__(self, drive_name: str, *args, **kwargs):
        super().__init__(*args, subset="all", **kwargs)
        self.elems = sorted(
            filter(lambda group: group.drive == drive_name and group.cam == 2, self.elems),
            key=lambda group: group.idx
        )

In [8]:
transform = ApplyToFeatures(
    Compose(
        [
            ToTensor(),
            RandomCrop([256, 512]),
        ]
    ),
    features=["img"],
)

ds = KittiDepthCompletionDataset(
    "kitti_raw_sync_rect_root",
    "kitti_depth_completion_root",
    load_stereo=False,
    load_sequence=3,
    transform=transform,
    download=True,  # download if not found
)

ERROR:root:missing data_depth_selection.zip: folder test_depth_completion_anonymous not found


ValueError: path kitti_depth_completion_root contains wrong data

In [18]:
# KITTI Test dataset and dataloader declaration
test_loader = torch.utils.data.DataLoader(
    datasets.Kitti('/home/aya/Kitti-ObjectDetection/KITTI-original/2D-objects/letf/testing/image_2/', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(), ])), batch_size=1, shuffle=True)

  0%|          | 0/12569945557 [00:00<?, ?it/s]

In [11]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [12]:
#test 
def test( model, device, test_loader, epsilon ):

    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in test_loader:

        # Send the data and label to the device
        data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == target.item():
            correct += 1
            # Special case for saving 0 epsilon examples
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [13]:
#run the attack
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(model, device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

NameError: name 'test_loader' is not defined

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier

In [6]:
# Step 2a: Define the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.01)

In [7]:
# Step 3: Create the ART classifier

classifier = PyTorchClassifier(
    model=pytorch_model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 1244, 375),
    nb_classes=9,
)

In [8]:
# Step 4: Train the ART classifier
classifier.fit(x_train, y_train, batch_size=8, nb_epochs=3)

: 

: 

In [ ]:
x_train[1].shape

(375, 1242, 3)

In [ ]:
x_train[1].shape

(1244, 375, 3)

In [3]:
# Load Image [0.0, 1.0], resize it then normalize it
x = np.asarray(Image.open('/home/aya/Desktop/Kitti-ObjectDetection/kitti-YOLOFormat/Adversarial.jpg').resize((32, 32))) / 255.0